In [15]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data
file_path_1 = '/Users/yashshah/Downloads/offers.csv'
file_path2 = '/Users/yashshah/Downloads/generators.csv'
file_path3 = '/Users/yashshah/Downloads/zones.csv'
file_path4 = '/Users/yashshah/Downloads/lines.csv'
file_path5 = '/Users/yashshah/Downloads/loops.csv'
offers_df = pd.read_csv(file_path_1)
generators_df = pd.read_csv(file_path2, index_col=0)
zones_df = pd.read_csv(file_path3, index_col=0)
lines_df = pd.read_csv(file_path4)
loops_df = pd.read_csv(file_path5, index_col=0)

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

def solve_economic_dispatch(path='.'):
    """
    Solves the ISO-NE economic dispatch problem using professor's specified notation:
    - x_gt: dispatched volume for generator g, tranche t
    - f_ij: flow from zone i to zone j
    - Q_gt: maximum quantity for generator g, tranche t
    - U_ij: capacity of line from i to j
    - D_i: demand at zone i
    - C_gt: cost of tranche t for generator g
    """
    try:
        # Load data files
        file_path_1 = '/Users/yashshah/Downloads/offers.csv'
        file_path2 = '/Users/yashshah/Downloads/generators.csv'
        file_path3 = '/Users/yashshah/Downloads/zones.csv'
        file_path4 = '/Users/yashshah/Downloads/lines.csv'
        file_path5 = '/Users/yashshah/Downloads/loops.csv'
        offers_df = pd.read_csv(file_path_1)
        generators_df = pd.read_csv(file_path2, index_col=0)
zones_df = pd.read_csv(file_path3, index_col=0)
lines_df = pd.read_csv(file_path4)
loops_df = pd.read_csv(file_path5, index_col=0)
        
        # Set multi-indices
        lines.set_index(['zone_from', 'zone_to'], inplace=True)
        offers.set_index(['generator', 'tranche'], inplace=True)
    except FileNotFoundError as e:
        print(f"Error loading files: {e}")
        return None

    # Create optimization model
    m = gp.Model("EconomicDispatch")
    
    # Decision variables (using exact notation from professor)
    x = m.addVars(offers.index, name="x_gt", lb=0)  # x_gt
    f = m.addVars(lines.index, name="f_ij", lb=-GRB.INFINITY)  # f_ij
    
    # Generator capacity constraints: x_gt ≤ Q_gt
    m.addConstrs(
        (x[g, t] <= offers.loc[(g, t), 'quantity']  # Q_gt
        for g, t in offers.index),
        name="generator_capacity"
    )
    
    # Create zone-to-generator mapping
    zone_to_gens = {zone: [] for zone in zones.index}
    for gen, zone in generators.itertuples():
        zone_to_gens[zone].append(gen)
    
    # Nodal balance constraints: ∑x_gt + ∑f_ji - ∑f_ij ≥ D_i
    for i in zones.index:
        # Sum of generation in zone i: ∑x_gt for gens in zone i
        total_gen = gp.quicksum(
            x[g, t]
            for g in zone_to_gens.get(i, [])
            for t in offers.loc[g].index
        )
        
        # Sum of inflows: f_ji where j is connected to i
        inflow = gp.quicksum(
            f[j, i] 
            for (j, to_zone) in lines.index
            if to_zone == i
        )
        
        # Sum of outflows: f_ij where i is connected to j
        outflow = gp.quicksum(
            f[i, j]
            for (from_zone, j) in lines.index
            if from_zone == i
        )
        
        # Constraint: total_gen + inflow - outflow ≥ D_i
        m.addConstr(
            total_gen + inflow - outflow >= zones.loc[i, 'load'],  # D_i
            name=f"nodal_balance_{i}"
        )
    
    # Kirchhoff's Voltage Law: ∑f_ij = 0 for each loop
    for loop in loops.index.unique():
        loop_lines = loops.loc[[loop]]
        loop_flow = gp.quicksum(
            f[i, j] if (i, j) in lines.index else -f[j, i]
            for i, j in zip(loop_lines['from'], loop_lines['to'])
        )
        m.addConstr(loop_flow == 0, name=f"loop_{loop}")
    
    # Line capacity constraints: f_ij ≤ U_ij and f_ij ≥ -U_ij
    m.addConstrs(
        (f[i, j] <= lines.loc[(i, j), 'capacity']  # U_ij
         for (i, j) in lines.index),
        name="line_capacity_upper"
    )
    m.addConstrs(
        (f[i, j] >= -lines.loc[(i, j), 'capacity']  # -U_ij
         for (i, j) in lines.index),
        name="line_capacity_lower"
    )
    
    # Objective: minimize ∑C_gt * x_gt
    total_cost = gp.quicksum(
        x[g, t] * offers.loc[(g, t), 'price']  # C_gt
        for g, t in offers.index
    )
    m.setObjective(total_cost, GRB.MINIMIZE)
    
    # Solve and process results
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        # Prepare results
        dispatch = pd.DataFrame([
            {
                'generator': g,
                'tranche': t,
                'dispatched': x[g, t].x,
                'price': offers.loc[(g, t), 'price']
            }
            for g, t in offers.index
            if x[g, t].x > 1e-6
        ])
        
        line_flows = pd.DataFrame([
            {
                'from': i,
                'to': j,
                'flow': f[i, j].x,
                'capacity': lines.loc[(i, j), 'capacity'],
                'congested': abs(abs(f[i, j].x) - lines.loc[(i, j), 'capacity']) < 1e-6
            }
            for (i, j) in lines.index
        ])
        
        nodal_prices = {
            i: m.getConstrByName(f"nodal_balance_{i}").Pi
            for i in zones.index
        }
        
        return {
            'model': m,
            'dispatch': dispatch,
            'flows': line_flows,
            'nodal_prices': pd.DataFrame.from_dict(nodal_prices, orient='index', columns=['price']),
            'total_cost': m.ObjVal
        }
    else:
        print("Optimization failed")
        return None

# Execute and display results
results = solve_economic_dispatch()
if results:
    print("Optimal Generation Dispatch:")
    print(results['dispatch'].to_string())
    
    print("\nLine Flows:")
    print(results['flows'].to_string())
    
    print("\nNodal Prices ($/MWh):")
    print(results['nodal_prices'].to_string())
    
    print("\nCongested Lines:")
    print(results['flows'][results['flows']['congested']].to_string())
    
    print(f"\nTotal Cost: ${results['total_cost']:,.2f}")

TypeError: cannot unpack non-iterable int object